# $$ Text-Data $$


**Our focus is on step 1 of stage 1: implementing the data sampling pipeline.**

### Pretraining Insight:
✅ Process text `one word` at a time <br/>
✅ Are trained on next-word prediction<br/>
- Models with:<br/>

✅ Millions to billions of parameters<br/>
→ Achieve impressive capabilities

### Why Data Preparation Is Needed
Before training an LLM:

✅ The training dataset must be prepared <br/>
Data must be:

- Tokenized
- Vectorized
- Sampled correctly

### What Is an Embedding?
Word embeddings convert words into numerical vectors that preserve semantic meaning.

### Why Embeddings Are Needed
- Neural networks cannot process raw text directly.
- Text is categorical, not numerical.
- Neural networks require:
✅ Continuous-valued numerical vectors.

### Embedding Models
Embeddings can be created using:
- A neural network layer.
- A pretrained embedding model

Different data types require: <br/>
✅ Different embedding models. <br/>
Text ≠ Audio ≠ Video

### Types of Text Embeddings
- Sentences
- Paragraphs
- Entire documents

Sentence and paragraph embeddings are commonly used in Retrieval-Augmented Generation (RAG) systems, which combine:
- Generation (text creation)
- Retrieval (searching external knowledge)

## Word2Vec
**One of the earliest and most popular word embedding methods is `Word2Vec`.**

How it works: <br/>
It trains a neural network to:
- Predict the context from a word.
- Or predict the word from its context.

**Main Idea:**

Words that appear in similar contexts tend to have similar meanings.

As a result, when visualized in 2D space:

> Similar words appear close together <br/>
> Related terms form clusters

<div style="text-align: center; margin-top: 20px;">
  <img 
    src="https://raw.githubusercontent.com/salavii/llm-from-scratch/main/images/2D Visualization of Word Embedding Space.png"
    style="width: 750px; border-radius: 10px; display: block; margin-left: auto; margin-right: auto;"
  >

  <p style="font-size: 16px; color: #333; font-weight: bold; margin-top: 10px;">
    This figure visualizes word embeddings projected into a two-dimensional space, where semantically similar words appear closer together. <br/>
    It demonstrates how relationships between concepts like animals, locations, and adjectives emerge in embedding space
  </p>
</div>


<br/>

## Embeddings in LLMs vs Word2Vec
In classical ML, embeddings can be generated using pretrained models such as Word2Vec.

In LLMs, embeddings are part of the model itself and are:
- Learned from scratch
- Updated during training

**Advantage:** LLM embeddings are optimized for:
- The specific task
- The specific dataset

**Real LLM embeddings are high-dimensional (hundreds to thousands)**

<div style="text-align: center; margin-top: 20px;">
  <img 
    src="https://raw.githubusercontent.com/salavii/llm-from-scratch/main/images/Text Tokenization to GPT Input Pipeline.png"
    style="width: 750px; border-radius: 10px; display: block; margin-left: auto; margin-right: auto;"
  >

  <p style="font-size: 16px; color: #333; font-weight: bold; margin-top: 10px;">
    This diagram illustrates the full pipeline from raw input text to token IDs and embeddings fed into a GPT-like decoder-only transformer. <br/>
    It shows how tokenization, embedding lookup, and postprocessing connect input text to generated output
  </p>
</div>


## Dataset

In [3]:
# Import the module for making HTTP requests and downloading files
import urllib.request
url = ("https://raw.githubusercontent.com/salavii/llm-from-scratch/refs/heads/main/data/the-verdict.txt")
file_path = "the-verdict.txt"

urllib.request.urlretrieve(url, file_path) # Download the file from the given URL and store it as "the-verdict.txt"


('the-verdict.txt', <http.client.HTTPMessage at 0x22afb284b90>)

In [9]:
# Open the text file in read mode using UTF-8 encoding
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Goal:
Tokenize a 20,479-character short story into:
- Individual words
- Special characters

So that these tokens can later be converted into `embeddings` for LLM training.

### How Do We Split the Text into Tokens?
To illustrate the basic idea of splitting text into tokens, we:
- Use Python’s **re (regular expression)** library.

Apply re.split to:
- Split text based on whitespace

This is only for demonstration.
Later, we will switch to a prebuilt tokenizer, and no regex knowledge will be required

In [15]:
import re
text = "Hello, world. This, is a test."

result = re.split(r'(\s)', text)        # Split the text based on whitespace characters
                                        # Parentheses ensure that the spaces are also included in the output
print(result)


['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


### Limitations of Simple Whitespace Tokenization
A simple whitespace-based tokenization:

>Separates most words correctly ✅ <br/>
>But still leaves punctuation attached to words ❌

### We do not convert all text to lowercase because:
Capitalization helps:
- Distinguish $proper$ $nouns$ from $common$ $nouns$.
- Understand sentence structure.
- Learn correct text generation with proper capitalization.

### Improved Tokenization Strategy

To improve tokenization, we split the text based on:
- Whitespace → \s
- Commas and periods → [,.]

In [16]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


A small remaining problem is that the list still includes whitespace characters. <br/>
Optionally, we can remove these redundant characters safely as follows:

In [17]:
result = [item for item in result if item.strip()]  # Remove whitespace-only tokens from the token list
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


### Handling Whitespace in Tokenization
When building a simple tokenizer, Whether to keep or remove whitespaces depends on the **application**.

#### Removing whitespaces:
- Reduces memory usage.
- Improves computational efficiency.

#### Keeping whitespaces:
- Preserves exact text structure.
- Is important for:
  > Programming languages <br/>
  > Indentation-sensitive data

In this project:

✅ Whitespaces are removed for simplicity. <br/>
✅ A later tokenizer will include whitespaces again.

### Extending the Tokenization Scheme
The current tokenizer works well for simple examples. <br/>
However, real-world text also contains:
- Question marks ?
- Quotation marks "
- Double dashes --

Other special characters

Therefore:

**The tokenization rules must be extended to correctly separate these symbols as well**

In [18]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Now that we have a basic tokenizer working, let’s apply it to Edith Wharton’s entire
short story:

In [20]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


# Converting tokens into token IDs
After tokenization, tokens are still represented as:
> Python strings (text)

However, LLMs: <br/>
Operate only on `numerical values.`

Therefore:<br/>
- Each token must be converted into a unique integer.
- This integer is called a **Token ID.**

This step is an $intermediate$ $stage$ before converting token IDs into:

**Embedding vectors**

### Building the Vocabulary
To map tokens to token IDs, we must first build a **vocabulary.**

A vocabulary:
- Contains all unique tokens
- Assigns: <br/>
> One unique integer To each word and special character.

<br/>

let’s create a list of all unique tokens and sort them
alphabetically to determine the vocabulary size:

In [29]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(f'vocabulary size is: {vocab_size}')

vocabulary size is: 1130


Now, we create the vocabulary and print its first 51 entries for illustration purposes.

In [34]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
